In [1]:
import importlib
import tensorflow as tf
import pandas as pd
import mpra_model
import h5py
importlib.reload(mpra_model)
import numpy as np
from tqdm import tqdm
import sklearn
from sklearn import model_selection
import scipy.stats
from sklearn.linear_model import Ridge
import os
os.environ['CUDA_VISIBLE_DEVICES']='1'

2024-02-21 21:48:37.246315: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


## Ridge/MLP model Train

In [2]:
ct_list = ['HepG2','K562']
model_list = []
LLM_list = []
perf_list = []
celltype_list = []
for model_name in ['hyena','gpn','dnabert2']:
    for ct in ct_list:
        print('Start')
        f = h5py.File('../data/lenti_MPRA_embed/'+model_name+'_'+ct+'.h5', 'r')
        x = f['seq']
        target = f['mean']
        x_train, x_test, y_train, y_test = model_selection.train_test_split(range(len(x)), range(len(target)), 
                                                                            test_size=0.1,random_state=42)
        if model_name == 'gpn':
            mean_embed = np.mean(x,axis=1)
            cls_embed = None
        else:
            mean_embed = np.mean(x[:,1:,:],axis=1)
            cls_embed = np.squeeze(x[:,:1,:])

        ## Ridge regression
        print('Ridge regression for CLS and Mean Embed')
        embed_model = Ridge(0.001).fit(mean_embed[np.sort(x_train)], target[np.sort(y_train)])

        LLM_list.append(model_name)
        model_list.append('Mean-embed-Ridge')
        perf_list.append(scipy.stats.pearsonr(embed_model.predict(mean_embed[np.sort(x_test)]),target[np.sort(y_test)])[0])
        celltype_list.append(ct)

        if cls_embed is not None:
            embed_model = Ridge(0.001).fit(cls_embed[np.sort(x_train)], target[np.sort(y_train)])

            LLM_list.append(model_name)
            model_list.append('CLS-Ridge')
            perf_list.append(scipy.stats.pearsonr(embed_model.predict(cls_embed[np.sort(x_test)]),target[np.sort(y_test)])[0])
            celltype_list.append(ct)

        ## MLP model
        print('MLP for mean embedding training...')
        model = mpra_model.rep_mlp(mean_embed.shape[1])
        optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)
        earlyStopping_callback = tf.keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)
        reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2,patience=5, min_lr=1e-8)
        model.compile(optimizer=optimizer,
                        loss='mean_squared_error',
                        metrics=['mse'])
        model.fit(
                mean_embed[np.sort(x_train)],target[np.sort(y_train)],
                epochs=100,
                batch_size=512,
                shuffle=True,
                validation_split=0.1,
                callbacks=[earlyStopping_callback,reduce_lr],
                verbose=0,)
        y_pred = model.predict(mean_embed[np.sort(x_test)])

        perf_list.append(scipy.stats.pearsonr(np.squeeze(y_pred),target[np.sort(y_test)])[0])
        LLM_list.append(model_name)
        celltype_list.append(ct)
        model_list.append('Mean-embed-MLP')
        if cls_embed is not None:
            print('MLP for cls training...')
            model = mpra_model.rep_mlp(cls_embed.shape[1])
            optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)
            earlyStopping_callback = tf.keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)
            reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2,patience=5, min_lr=1e-8)
            model.compile(optimizer=optimizer,
                            loss='mean_squared_error',
                            metrics=['mse'])
            model.fit(
                    cls_embed[np.sort(x_train)],target[np.sort(y_train)],
                    epochs=100,
                    batch_size=512,
                    shuffle=True,
                    validation_split=0.1,
                    callbacks=[earlyStopping_callback,reduce_lr],
                    verbose=0,)
            y_pred = model.predict(cls_embed[np.sort(x_test)])

            perf_list.append(scipy.stats.pearsonr(np.squeeze(y_pred),target[np.sort(y_test)])[0])
            LLM_list.append(model_name)
            celltype_list.append(ct)
            model_list.append('CLS-MLP')

        del(model)
        tf.keras.backend.clear_session()
        

Start
Ridge regression for CLS and Mean Embed
MLP for mean embedding training...


2024-02-20 21:05:55.552442: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1635] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 78973 MB memory:  -> device: 0, name: NVIDIA A100 80GB PCIe, pci bus id: 0000:47:00.0, compute capability: 8.0
/home/ztang/.conda/envs/jax_tf/lib/python3.9/site-packages/keras/initializers/initializers.py:120: UserWarning: The initializer RandomNormal is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initalizer instance more than once.
  warnings.warn(
2024-02-20 21:06:27.118235: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:637] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2024-02-20 21:06:27.120403: I tensorflow/compiler/xla/service/service.cc:169] XLA service 0x7f06a7030b00 initialized for platform CUDA (this does n

438/438 [==============================] - 0s 586us/step
MLP for cls training...
438/438 [==============================] - 0s 607us/step
Start
Ridge regression for CLS and Mean Embed
MLP for mean embedding training...


/home/ztang/.conda/envs/jax_tf/lib/python3.9/site-packages/keras/initializers/initializers.py:120: UserWarning: The initializer RandomNormal is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initalizer instance more than once.
  warnings.warn(


708/708 [==============================] - 0s 597us/step
MLP for cls training...
708/708 [==============================] - 0s 579us/step
Start
Ridge regression for CLS and Mean Embed
MLP for mean embedding training...


...

438/438 [==============================] - 0s 632us/step
MLP for cls training...
438/438 [==============================] - 0s 614us/step
Start
Ridge regression for CLS and Mean Embed


/home/ztang/.conda/envs/jax_tf/lib/python3.9/site-packages/sklearn/linear_model/_ridge.py:216: LinAlgWarning: Ill-conditioned matrix (rcond=5.19195e-08): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
/home/ztang/.conda/envs/jax_tf/lib/python3.9/site-packages/sklearn/linear_model/_ridge.py:216: LinAlgWarning: Ill-conditioned matrix (rcond=8.71962e-09): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T


MLP for mean embedding training...


/home/ztang/.conda/envs/jax_tf/lib/python3.9/site-packages/keras/initializers/initializers.py:120: UserWarning: The initializer RandomNormal is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initalizer instance more than once.
  warnings.warn(


708/708 [==============================] - 0s 612us/step
MLP for cls training...
708/708 [==============================] - 0s 618us/step


In [3]:
perf_df = pd.DataFrame({'LLM':LLM_list,'Model':model_list,'Performance':perf_list,'Cell Type':celltype_list})

In [5]:
perf_df.to_csv('./results/LLM_baseline.csv',index=False)

## CNN Model Train

In [ ]:
data_file = '../data/lenti_MPRA_embed/sei_K562.h5'
model_file = '../model/lenti_MPRA/lenti_MPRA_embed/K562/sei.h5'

In [10]:
cnn_config = {
    'activation':'exponential',
    'reduce_dim': 128,
    'conv1_filter':196,
    'conv1_kernel':7,
    'dropout1':0.2,
    'res_filter':5,
    'res_layers':3,
    'res_pool':5,
    'res_dropout':0.2,
    'conv2_filter':256,
    'conv2_kernel':7,
    'pool2_size':4,
    'dropout2':0.2,
    'dense':512,
    'dense2':256,
    'l_rate':0.0001
}

file = h5py.File(data_file,'r')
seq = file['seq'][()]
target = file['mean'][()]
x_train,x_test,y_train,y_test=model_selection.train_test_split(seq,target,random_state=42,test_size=0.1)

model = mpra_model.rep_cnn(seq[0].shape,cnn_config)
optimizer = tf.keras.optimizers.Adam(learning_rate=cnn_config['l_rate'])
loss = tf.keras.losses.MeanSquaredError()
model.compile(optimizer=optimizer,loss=loss,metrics=['mse'])
earlyStopping_callback = tf.keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)

reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss', factor=0.2,
        patience=5, min_lr=1e-8)
checkpoint = tf.keras.callbacks.ModelCheckpoint(
                                    model_file,
                                    monitor='val_loss',
                                    save_best_only=True,
                                    mode = 'min',
                                    save_freq='epoch',)
model.fit(
        x_train,y_train,
        epochs=100,
        batch_size=512,
        shuffle=True,
        verbose=0,
        validation_split=0.1,
        callbacks=[earlyStopping_callback,reduce_lr,checkpoint])

y_pred = model.predict(x_test)
print(scipy.stats.pearsonr(np.squeeze(y_pred),np.squeeze(y_test)))

708/708 [==============================] - 1s 1ms/step
PearsonRResult(statistic=0.7715274284093354, pvalue=0.0)


In [3]:
## Embedding results
for model_name in ['hyena']:
    for ct in ['HepG2','K562']:
        f = h5py.File('../data/lenti_MPRA_embed/'+model_name+'_'+ct+'.h5', 'r')
        x = f['seq']
        y = f['mean']
        x_train, x_test, y_train, y_test = model_selection.train_test_split(range(len(x)), range(len(y)), 
                                                                            test_size=0.1,random_state=42)
        x_test = x[np.sort(x_test)]
        y_test = y[np.sort(y_test)]
       
        model = tf.keras.models.load_model('../model/lenti_MPRA/lenti_MPRA_embed/'+ct+'/'+model_name+'.h5')
        y_pred = model.predict(x_test)
        pr = scipy.stats.pearsonr(np.squeeze(y_pred),y_test)[0]
        print(pr)

2024-02-20 16:20:49.548413: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1635] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 78973 MB memory:  -> device: 0, name: NVIDIA A100 80GB PCIe, pci bus id: 0000:47:00.0, compute capability: 8.0
/home/ztang/.conda/envs/jax_tf/lib/python3.9/site-packages/keras/initializers/initializers.py:120: UserWarning: The initializer RandomNormal is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initalizer instance more than once.
  warnings.warn(
2024-02-20 16:20:52.849575: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:637] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2024-02-20 16:20:52.974980: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:424] Loaded cuDNN version 8800


438/438 [==============================] - 2s 2ms/step
0.5626900592550687
708/708 [==============================] - 1s 2ms/step
0.6613938648554784
